In [17]:
import json
import pandas as pd
import requests
import os

In [10]:
VA = pd.read_csv("VA_addresses.csv").drop(columns=["Unnamed: 0"])

In [11]:
VA.loc[VA['PC'] == "8448GW"]

,HUISNR,PC
225,104,8448GW


In [14]:
VA_test = VA[224:227]

In [15]:
VA_test

,HUISNR,PC
224,209,8096BJ
225,104,8448GW
226,2,8448CN


In [25]:
columns = ["huisnummer","postcode","nrAanduiding","vblObject","inOnderzoek"]
if os.path.isfile("kadaster_ids.csv"):
    kadaster_ids = pd.read_csv("kadaster_ids.csv",dtype=object)
else:
    kadaster_ids = pd.DataFrame(columns=columns)

In [30]:
kadaster_ids

,huisnummer,postcode,nrAanduiding,vblObject,inOnderzoek
0,209,8096BJ,0269200000008628,0269010000008628,False
0,2,8448CN,0074200000373763,0074010000373764,False


In [27]:
def getResponse(PC, HUISNR):
    url = "https://bag.basisregistraties.overheid.nl/api/v1/nummeraanduidingen?postcode={}&huisnummer={}".format(PC,HUISNR)

    payload = {}
    headers = {
      'x-api-key': '593f1c08-4eac-4f5b-8a35-59d64883e2b2'
    }

    response = requests.request("GET", url, headers=headers, data = payload)
    return response

In [28]:
def getInfo(json_response):
    huisnummer = json_response["_embedded"]["nummeraanduidingen"][0]["huisnummer"]
    postcode = json_response["_embedded"]["nummeraanduidingen"][0]["postcode"]
    nrAanduiding = json_response["_embedded"]["nummeraanduidingen"][0]["identificatiecode"]
    vblObject = json_response["_embedded"]["nummeraanduidingen"][0]["_links"]["adresseerbaarObject"]["href"].rpartition("/")[-1]
    inOnderzoek = json_response["_embedded"]["nummeraanduidingen"][0]["_embedded"]["geldigVoorkomen"]["inOnderzoek"]
    
    df = pd.DataFrame(data = [[huisnummer,postcode,nrAanduiding,vblObject,inOnderzoek]],columns=columns)
    return df

In [29]:
for index, row in VA_test.iterrows():
    HUISNR = row["HUISNR"]
    PC = row["PC"]
    response = getResponse(PC, HUISNR)
    json_response = json.loads(response.text)
    try: 
        kadaster_ids = kadaster_ids.append(getInfo(json_response))
    except IndexError:
        print("out of range")

out of range


In [69]:
kadaster_ids.to_csv("kadaster_ids.csv", index=None)